In [1]:
!pip install -U transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.0
    Uninstalling transformers-4.36.0:
      Successfully uninstalled transformers-4.36.0


In [8]:
!pip install -U accelerate



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00


























In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # or another variant like "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [22]:
class EssaysDataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.block_size = block_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]["text"]
        title = self.data.iloc[idx]['title']
        subject = self.data.iloc[idx]["subject"]

        input_text = f"Subject: {subject}\nTitle: {title}\nText: {text}"

        # Tokenize and truncate the input text
        input_ids = self.tokenizer.encode(
            input_text, max_length=self.block_size, truncation=True
        )

        return input_text # {"input_ids": torch.tensor(input_ids, dtype=torch.long)}

In [24]:
dataset.__getitem__(1)

'Subject: News\nTitle:  Drunk Bragging Trump Staffer Started Russian Collusion Investigation\nText: House Intelligence Committee Chairman Devin Nunes is going to have a bad day. He s been under the assumption, like many of us, that the Christopher Steele-dossier was what prompted the Russia investigation so he s been lashing out at the Department of Justice and the FBI in order to protect Trump. As it happens, the dossier is not what started the investigation, according to documents obtained by the New York Times.Former Trump campaign adviser George Papadopoulos was drunk in a wine bar when he revealed knowledge of Russian opposition research on Hillary Clinton.On top of that, Papadopoulos wasn t just a covfefe boy for Trump, as his administration has alleged. He had a much larger role, but none so damning as being a drunken fool in a wine bar. Coffee boys  don t help to arrange a New York meeting between Trump and President Abdel Fattah el-Sisi of Egypt two months before the election.

In [26]:
from tqdm.notebook import tqdm

file_path = "/kaggle/working/output_file.txt"

# Open the file in write mode
with open(file_path, 'w') as file:
    # Iterate through the sentences and write each one to the file
    for i in tqdm(range(dataset.__len__())):
        text = dataset.__getitem__(i)
        file.write(text + '\n')  # Add a newline character to separate sentences

  0%|          | 0/23481 [00:00<?, ?it/s]

In [27]:
# Set your paths and filenames
# train_data_path = "/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv"
output_dir = "/kaggle/output/fine-tuned/model"

# Load your dataset using the custom class
# dataset = EssaysDataset(
#     file_path=train_data_path,
#     tokenizer=tokenizer,
#     block_size=256,
# )

train_data_path = "/kaggle/working/output_file.txt"

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_data_path,
    block_size=128,
)

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [28]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/output/fake_news_model",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust as needed
    per_device_train_batch_size=4,  # Adjust based on your GPU memory
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine_tuned_fake_news_model")
tokenizer.save_pretrained("fine_tuned_fake_news_model")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,3.592200
1000,3.489200
1500,3.453300
2000,3.428600
2500,3.426200
3000,3.398700
3500,3.397100
4000,3.379700
4500,3.364900
5000,3.363200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


('fine_tuned_fake_news_model/tokenizer_config.json',
 'fine_tuned_fake_news_model/special_tokens_map.json',
 'fine_tuned_fake_news_model/vocab.json',
 'fine_tuned_fake_news_model/merges.txt',
 'fine_tuned_fake_news_model/added_tokens.json')

In [30]:
ls '/kaggle/working/fine_tuned_fake_news_model'

config.json             merges.txt         special_tokens_map.json  vocab.json
generation_config.json  model.safetensors  tokenizer_config.json


In [35]:
# Set the path to your fine-tuned model
# fine_tuned_model_path = "path/to/fine-tuned/model"

# Load the fine-tuned model and tokenizer
# model = GPT2LMHeadModel.from_pretrained(fine_tuned_model_path)
# tokenizer = GPT2Tokenizer.from_pretrained(fine_tuned_model_path)

# Set the model to evaluation mode
model.eval()

# Provide a topic as input
topic = "Politics"

# Generate text based on the input topic
input_text = f"Topic: {topic}\nNews:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate text using the model
output = model.generate(input_ids, max_length=200, num_beams=1, temperature=0.8)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated essay
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Topic: Politics
News:  Trump’s ‘Fake News’ Campaign Is Going To Be A Huge Disaster
Text: Donald Trump s campaign is going to be a disaster. The Republican nominee has been using his Twitter account to attack the media and the American people for months now. He s been using his Twitter account to attack the media and the American people for months now. He s been using his Twitter account to attack the media and the American people for months now.  Donald J. Trump (@realDonaldTrump) August 9, 2016Trump s campaign is going to be a disaster. The Republican nominee has been using his Twitter account to attack the media and the American people for months now. He s been using his Twitter account to attack the media and the American people for months now.  Donald J. Trump (@realDonaldTrump) August 9, 2016Trump s campaign is going to be a disaster. The Republican nominee has been using his Twitter account to attack the media and the American people for
